In [ ]:
# Version 25-04-07
# - Changed stratification method to bins, which is suited for regression
# - Defined functions for code organization and conciseness

In [ ]:
# Library imports
from category_encoders.target_encoder import TargetEncoder
from contextlib import suppress
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from IPython.display import display
from sklearn.metrics import r2_score, PredictionErrorDisplay
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
from xgboost import plot_importance, XGBRegressor
import time
from datetime import datetime

In [ ]:
# Record and display execution start time
start_time = time.time()
now = datetime.now()
print(now.strftime("%d/%m/%Y %H:%M:%S"))

In [ ]:
def dataset(target, min_target_counts=1, cols_to_drop=[]):
    """Read and preprocess a .CSV file into a filtered pandas DataFrame.

    Args:
        target (string): Name of the target property (must match a column name
            in the .CSV file)
        min_target_counts (int | float): Minimum count threshold for target
            values. Rows with values appearing fewer times are dropped.
    
    Returns:
        pandas.DataFrame: Filtered dataset with only relevant columns. Rows
            with values appearing fewer times are dropped.
    
    Notes:
        The .CSV file path is hardcoded in this version and not
            user-configurable
        XGBoost is sparsity-aware. No need to drop missing values.
    """

    # Get .CSV file path
    main_path = str(os.getcwd())
    file = main_path + '\\Entradas\\degrad_dataset.csv'

    # Read .CSV as pandas dataframe
    df = pd.read_csv(file, delimiter=';')

    # Drop columns that are not important for the analysis
    df = df.drop(columns=cols_to_drop)

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Filters target property values that appear less than 4 times
    value_counts = df[target].value_counts()
    valid_values = value_counts[value_counts > min_target_counts - 1].index
    df = df[df[target].isin(valid_values)]

    # Encoding categorical features
    pass

    return df

In [ ]:
def get_abs_error(y_test, y_pred):  
    """Calculate the maximum signed absolute error between predicted and actual
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: Maximum signed absolute error (y_pred - y_test), rounded to
                3 decimals.
        
        Notes:
            Positive values indicate overprediction, negative underprediction.
            Both inputs must have the same length.    
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_abs_error(y_test, y_pred)
        -0.4
    """
    
    y_test = y_test.tolist()
    a = []
    for i in range(len(y_test)):
        a.append((y_pred[i] - y_test[i]))
        
    abs_error=0
    if max(abs(i) for i in a) == max(a):
        abs_error = max(a)
    else:
        abs_error = min(a)

    return round(abs_error, 3)

In [ ]:
def get_rel_error(y_test, y_pred):
    """Calculate the maximum signed relative error between predicted and actual
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: Maximum signed relative error ((y_pred - y_test) / y_test),
                rounded to 3 decimals.
        
        Notes:
            Positive values indicate overprediction, negative underprediction.
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_rel_error(y_test, y_pred)
        -0.167
    """
    
    y_test = y_test.tolist()
    a = []
    for i in range(len(y_test)):
        a.append((y_pred[i] - y_test[i])/y_test[i])
        
    rel_error=0
    if max(abs(i) for i in a) == max(a):
        rel_error = max(a)
    else:
        rel_error = min(a)

    return round(rel_error, 3)

In [ ]:
def get_mae(y_test, y_pred):
    """Calculate the mean absolute error (MAE) between predicted and actual values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: MAE (Σ(|y_pred - y_test|) / len(y_test)), rounded to 3 decimals.
        
        Note:
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_mae(y_test, y_pred)
        0.233
    """
    
    y_test = y_test.tolist()
    a = 0
    for i in range(len(y_test)):
        a = a + abs(y_pred[i] - y_test[i])
        
    mae = a / len(y_test)

    return round(mae, 3)

In [ ]:
def get_rmse(y_test, y_pred):
    """Calculate the root mean squared error (RMSEA) between predicted and
        actual values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: RMSE ((Σ(|y_pred - y_test|)² / len(y_test))^0.5), rounded
                to 3 decimals.
        
        Note:
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_rmse(y_test, y_pred)
        0.265
    """
    
    y_test = y_test.tolist()
    a = 0
    for i in range(len(y_test)):
        a = a + (y_pred[i] - y_test[i]) ** 2
        
    rmse = np.sqrt(a / len(y_test))

    return round(rmse, 3)

In [ ]:
def get_r2(y_test, y_pred):
    """Calculate the coefficient of determination (or R² score) of predicted
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: R² score (1 - RSS/TSS), rounded to 3 decimals.
        
        Note:
            RSS: Residual sum of squares (Σ(y_pred - y_test)²).
            TSS: Total sum of squares (Σ(mean - y_test)²), i.e., variability of
                the actual data.
            An R² score equal to 1 indicates a perfect fit between predicted and
                actual values. An R² equal to 0 means the prediction is no better
                than the mean. A negative R² indicates the prediction is worse
                than the mean.
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_r2(y_test, y_pred)
        0.978
    """
    
    y_test = y_test.tolist()
    mean = sum(y_test) / len(y_test)
    rss = 0  # Residual sum of squares
    tss = 0  # Total sum of squares
    for i in range(len(y_test)):
        rss = rss + (y_pred[i] - y_test[i]) ** 2
        tss = tss + (mean - y_test[i]) ** 2

    r2 = 1 - rss / tss

    return round(r2, 3)

In [186]:
def run(n_splits=10):
    """...
    """

    for i in range(n_splits):
        print(i)

run()

    

0
1
2
3
4
5
6
7
8
9


In [ ]:
abs_error_list = []
max_rel_error_list = []
MAE_list = []
RMSE_list = []
R2_list = []

In [ ]:
target='Tensile modulus retention'
min_target_counts = 1
cols_to_drop = ['Author',
                'Isophtalic polyester resin',
                'Orthophtalic polyester resin',
                'Vinylester resin',
                'Phenolic resin',
                'Epoxy resin',
                'Glass fiber',
                'Carbon fiber',
                'Pultrusion',
                'Hand lay-up',
                'Filament winding',
                'VARTM',
                'Coupon descr.',
                'Aging effect',
                'Steady condition',
                'Cyclic condition',
                'Immersion',
                'Moisture',
                'Presence of salts',
                #'Relative humidity',
                #'Sustained loading',
                'Exposure time (hours)',
                'Min. exposure temperature (ºC)',
                'Max. exposure temperature (ºC)'#,
                #'Residual tensile modulus (GPa)',
                #'Unaged tensile modulus (GPa)',
                ]

df = dataset(target=target,
             min_target_counts=min_target_counts,
             cols_to_drop=cols_to_drop)

display(df.head())
df.info()
df[target].value_counts()

### Random state = 0

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=0)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 0.3, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
abs_error = get_abs_error(y_test, y_pred)

abs_error_list.append(abs_error)
print("Maximum error = ", round(abs_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

For tree model, "Importance type" can be defined as:

- ‘weight’: the number of times a feature is used to split the data across all trees.
- ‘gain’: the average gain across all splits the feature is used in.
- ‘cover’: the average coverage across all splits the feature is used in.
- ‘total_gain’: the total gain across all splits the feature is used in.
- ‘total_cover’: the total coverage across all splits the feature is used in.

In [ ]:
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

with suppress(ValueError): 
    plot_importance(xgboost_model)

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=0)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 1

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=1)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 2

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=2)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 3

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=3)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         )) 
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

### Random state = 4

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=4)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 5

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=5)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 6

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=6)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 7

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=7)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         ))
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 8

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=8)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         )) 
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)
    #plt.savefig(str(os.getcwd()) + '\\Saídas\\tree_' + now.strftime("%d-%m-%Y_%H-%M-%S") + '.jpg')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Random state = 9

In [ ]:
# Separate independent variables from target
X = df.drop(columns='Tensile modulus retention')
y = df['Tensile modulus retention']

# Bin the continuous target into categories for stratification
bins = pd.qcut(y, q=5, duplicates='drop')  # You can change q to another number (e.g. 5 or 8) if needed

# Split data in training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=9)

In [ ]:
# List of tuples with transformations to be passed to the training pipeline
estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBRegressor(random_state=0, booster='gbtree'
                         , base_score=0.9#, n_estimators=2000
                         , eval_metric=r2_score#, early_stopping_rounds=50, eval_set=[(X_test, y_test)]
                        #  , objective='reglinear',
                         )) 
]
pipe = Pipeline(steps=estimators)

In [ ]:
# Search range of the hyperparameters to be tuned
search_space = {
    # 'clf__base_score': Real(0.0, 1000.0),
    'clf__max_depth': Integer(2, 8),
    'clf__n_estimators': Integer(50, 2000),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.6, 1.0),
    'clf__colsample_bytree': Real(0.6, 1.0),
    # 'clf__colsample_bylevel': Real(0.1, 1.0),
    # 'clf__colsample_bynode': Real(0.1, 1.0),
    'clf__reg_alpha': Real(0.0, 1.0), # L1 regularization
    # 'clf__reg_lambda': Real(0.0, 5.0),
    # 'clf__gamma': Real(0.0, 2.0),
    # 'clf__min_child_weight': Real(1.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=60
, scoring='r2', error_score="raise"
 , random_state=0, return_train_score=True, n_jobs=6, n_points=1) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_score_

In [ ]:
opt.best_params_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
print("y_test")
y_test.values

In [ ]:
y_pred = opt.predict(X_test)

print("y_pred")
y_pred

In [ ]:
# Maximum error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i]))
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_error_list.append(max_error)
print("Maximum error = ", round(max_error, 3))

In [ ]:
# Maximum relative error of prediction in relation to the test data
a = y_test.tolist()
b = []
for i in range(len(a)):
    b.append((a[i] - y_pred[i])/a[i])
    
if abs(max(b)) > abs(max(b)):
    max_error = max(b)
else:
    max_error = min(b)

max_rel_error_list.append(max_error)
print("Maximum relative error = ", 100 * round(max_error, 2), " %", sep='')

In [ ]:
# Mean absolute error (MAE) of prediction in relation to the test data
a = y_test.tolist()
avg = sum(a) / len(a)
b = 0
c = 0
for i in range(len(a)):
    b = b + abs(a[i] - y_pred[i])
    c = c + abs(a[i] - avg)
    
MAE_pred = b / len(a)
MAE_avg = c / len(a)
MAE_list.append(MAE_pred)
print("Prediction MAE =", MAE_pred)
print("Average MAE =", MAE_avg)

In [ ]:
# Root mean squared error (RMSE) of the prediction

b = 0
c = 0
for i in range(len(a)):
    b = b + (a[i] - y_pred[i]) ** 2
    c = c + (a[i] - avg) ** 2
    
RMSE_pred = np.sqrt(b / len(a))
RMSE_avg = np.sqrt(c / len(a))
RMSE_list.append(RMSE_pred)
print("Prediction RMSE =", RMSE_pred)
print("Average RMSE =", RMSE_avg)

In [ ]:
# R² score of the prediction

a = y_test.tolist()
RSS = 0             # Sum of the square of the residuals of the prediction
TSS = 0             # Sum of the square of the residuals of the average
for i in range(len(a)):
    RSS = RSS + (a[i] - y_pred[i]) ** 2
    TSS = TSS + (a[i] - avg) ** 2

R2 = 1 - RSS / TSS
R2_list.append(R2)
print("Average =", avg, "\nR² =", R2)

In [ ]:
display = PredictionErrorDisplay.from_predictions(y_test, y_pred, kind="actual_vs_predicted")
display.plot()
plt.show()

In [ ]:
# importance_type = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]

xgboost_model.get_booster().get_score(importance_type='weight')

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_tree(xgboost_model, ax=ax, num_trees=500)

In [ ]:
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(25,10))
    ax = fig.add_subplot(111)
    xgb.plotting.plot_importance(xgboost_model, ax=ax)

### Score lists

In [ ]:
print("Max. error:", max_error_list)
print("\nMax. relative error:", max_rel_error_list)
print("\nMAE:", MAE_list)
print("\nRMSE:", RMSE_list)
print("\nR²:", R2_list)

In [ ]:
print("\nMax. error =", round(min(max_error_list), 2))
print("Min. error =", round(max(max_error_list), 2))
print("Max. error =", round(np.mean(max_error_list), 2), "±", round(np.std(max_error_list), 2))

print("\nMax. relative error = ", 100 * round(min(max_rel_error_list), 2), " %", sep='')
print("Min. relative error = ", 100 * round(max(max_rel_error_list), 2), " %", sep='')
print("Max. relative error = ", 100 * round(np.mean(max_rel_error_list), 2), " ± ", 100 * round(np.std(max_rel_error_list), 2), " %", sep='')

print("\nMax. MAE =", round(max(MAE_list), 2))
print("Min. MAE =", round(min(MAE_list), 2))
print("MAE =", round(np.mean(MAE_list), 2), "±", round(np.std(MAE_list), 2))

print("\nMax. RMSE =", round(max(RMSE_list), 2))
print("Min. RMSE =", round(min(RMSE_list), 2))
print("RMSE =", round(np.mean(RMSE_list), 2), "±", round(np.std(RMSE_list), 2))

print("\nMax. R² =", round(max(R2_list), 2))
print("Min. R² =", round(min(R2_list), 2))
print("R² =", round(np.mean(R2_list), 2), "±", round(np.std(R2_list), 2))

In [ ]:
# Fim do processamento
now = datetime.now()

print(now.strftime("%d/%m/%Y %H:%M:%S"))
print("\nTempo de processamento: %s segundos\n" % (round((time.time() - start_time), 2)))